In [ ]:

# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.linear_model import LinearRegression
from sklearn import preprocessing
import datetime as dt
# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

from subprocess import check_output
print(check_output(["ls", "../input"]).decode("utf8"))

# Any results you write to the current directory are saved as output.
#read from fundamentals.csv
fundData = pd.read_csv("../input/fundamentals.csv")
priceData = pd.read_csv("../input/prices.csv")

#rename column name to prepare for merging
fundData = fundData.rename(columns={'Period Ending': 'date'})
fundData = fundData.rename(columns={'Ticker Symbol': 'symbol'})

#print(fundData.head())
#print("________")
#print(priceData.head())

#innerjoin the fund table to the price table, since we only data that is complete 
mergedTable = pd.merge(fundData, priceData, how='inner', on=['date', 'symbol'])
#fill up all the NA fields
mergedTable = mergedTable.fillna(0)

le = preprocessing.LabelEncoder()

#print("________")
#print(mergedTable.head())

#then split into predictors and target sets
targetSet = mergedTable.iloc[:,-5:-1]
predictorSet = mergedTable.iloc[:,:-5]

#encode the ticker string to integer
predictorSet["symbol"] = le.fit_transform(predictorSet["symbol"])
#encode date
predictorSet["date"] = pd.to_datetime(predictorSet['date'])
predictorSet['epoch'] = (predictorSet['date'] - dt.datetime(1970,1,1)).dt.total_seconds()
#then drop date column
predictorSet = predictorSet.drop(['date'], axis=1)

trainTargetData = targetSet[:1000]
trainPredictorData = predictorSet[:1000]
testPredictorData = predictorSet[1000:]
print(trainTargetData.head())
print(trainPredictorData.head())
#print(targetSet.shape)
#print(predictorSet.shape)

#print("Contains NA values:"+str(trainData.isnull().any().any()))

#finally..
lm = LinearRegression()
lm.fit(trainPredictorData,trainTargetData) 
predictions = lm.predict(testPredictorData)
#print("training set size:"+trainData.shape)
#print("training set size:"+testData.shape)
#print(trainData)